In [ ]:
!pip3 install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 19.4 MB/s eta 0:00:00


In [ ]:
import h5py
import pandas as pd
import json
import numpy as np
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as ssp
from sklearn.preprocessing import normalize
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import pickle
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import itertools
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix
from collections import Counter
import pickle
from sklearn.utils import shuffle
from os.path import join as pjoin
from tqdm import tqdm
import os
import joblib
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import importlib
from sklearn.preprocessing import normalize as sk_normalize
import pytorch_lightning as pl
from os.path import join as pjoin
from scipy import sparse

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/experiment_design2/ex2/kakao-recoteam-recsys-2022-challenge/mlp

Mounted at /content/drive
/content/drive/MyDrive/experiment_design2/ex2/kakao-recoteam-recsys-2022-challenge/mlp


In [ ]:
torch.cuda.set_device(0)

In [ ]:
fname = "procseed_data_with_aug"

In [ ]:
lm_feats = joblib.load('%s/lm_feats' % fname)

In [ ]:
tr_ds = joblib.load('%s/tr_ds' % fname)
val_ds = joblib.load("%s/val_ds" % fname)
id2idx = joblib.load('%s/id2idx' % fname)
idx2id = {y:x for (x,y) in id2idx.items()}
n_items = len(id2idx)

In [ ]:
val_csv = joblib.load("%s/val_csv" % fname)
tr_csv = joblib.load('%s/tr_csv' % fname)

In [ ]:
lr = 1.5 * 1e-4
# 1, 3, 7, 17, 7919, 11810044, 2345243545678234
torch.manual_seed(2345243545678234)
import mlps
importlib.reload(mlps)

def get_model(dim=256):
    return mlps.MLP2(lm_feats.toarray(), tr_ds.matrices,
                    tr_ds.categoricals,
                    tr_ds.tr_scalar,
                    n_items=n_items,
                    dim=np.random.choice([256]),
                    layer_dim=np.random.choice([3000]),
                    dropout=np.random.choice([0.3, 0.35]),
                    use_cat=False)

mlp_models = [get_model() for x in range(10)]

def get_opts(model):
    wd = np.random.choice([1e-3])
    return torch.optim.Adam(model.parameters(),  lr=lr, weight_decay=wd)

opts = [get_opts(model) for model in mlp_models]

for m in mlp_models:
    m.cuda()
    m.train()

lf = torch.nn.CrossEntropyLoss()


item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_l

이렇게 3개고 last view만 aug해줬을 때 성능이 0.1928이 나옴?
0.3, 0.35, 0.3
5e-3, 5e-3 3e-3

In [ ]:
tr_dl = DataLoader(tr_ds,
                   batch_size=256, shuffle=False,
                   num_workers=12, prefetch_factor=8,
#                    persistent_workers=True,
                   pin_memory=True)

In [ ]:
def validate(model, val_ds, use_filter=True, skip_val_only=False):
    hr = []
    model = model.eval()
    s = 0
    tot_mrr = []
    ii = 0
    for input in DataLoader(val_ds, batch_size=250, shuffle=False):
        (target_idx, _), (ret_mat, ret_cat, ret_scalar) = input
        for k in ret_mat:
            ret_mat[k] = ret_mat[k].cuda()
        for k in ret_cat:
            ret_cat[k] = ret_cat[k].cuda()
#         ret_cat = {}
        ret_scalar = ret_scalar.cuda()
        ret = model.forward(ret_mat, ret_cat, ret_scalar,)
        ret[ret_mat['item_bow'].bool()] = -1000000.0
#         ret[:, ~z] = -10000000.0
        top_rec = (-ret).argsort(-1)[:, :100].cpu()


        if skip_val_only:
            top_rec = top_rec[target_idx != 29999]
            target_idx = target_idx[target_idx != 29999]

        mrr = (top_rec == target_idx.unsqueeze(1)).float().numpy()
        mrr = (mrr / np.expand_dims(np.arange(1, 1 + 100), 0))
        mrr = mrr.sum(-1)
        tot_mrr.extend(mrr.tolist())
#         break
    model = model.train()
    return np.mean(tot_mrr)

In [ ]:
def flatten(x):
    ret = []
    for k in x:
        ret.extend(k)
    return ret

In [ ]:
def validate_models(models, val_ds, use_filter=True, skip_val_only=False, r=None):
    hr = []
    for model in models:
        model = model.eval()
    s = 0
    tot_mrr = []
    ii = 0
    for input in DataLoader(val_ds, batch_size=250, shuffle=False):
        (target_idx, _), (ret_mat, ret_cat, ret_scalar) = input
        for k in ret_mat:
            ret_mat[k] = ret_mat[k].cuda()
        for k in ret_cat:
            ret_cat[k] = ret_cat[k].cuda()
        ret_scalar = ret_scalar.cuda()
        rret = []
        for model in models:
            ret = model.forward(ret_mat, ret_cat, ret_scalar,)
            rret.append(ret)
        ret = torch.stack(rret)
        ret = torch.mean(ret, 0)
        ret[ret_mat['item_bow'].bool()] = -10000.0
        if r is not None:
            print(r.shape)
            ret[:, ~r] = -100000.0
        top_rec = (-ret).argsort(-1)[:, :100].detach().cpu()

#         if skip_val_only:
#         top_rec = top_rec[target_idx != 29999]
#         target_idx = target_idx[target_idx != 29999]

        mrr = (top_rec == target_idx.unsqueeze(1)).float().numpy()
        hitst = mrr[:, :5].sum(-1)
        mrr = (mrr / np.expand_dims(np.arange(1, 1 + 100), 0)).sum(-1)
        tot_mrr.extend(mrr.tolist())
        hr.extend(hitst)
    for model in models:
        model = model.train()
    return np.mean(tot_mrr), np.mean(hr)


In [ ]:
validate_models(mlp_models, val_ds)

(0.00020076849812597882, 7.3514995e-05)

In [ ]:
loss_ = np.log(n_items)
max_mrr = -1
for epoch in range(30):
    it = 0
    iidx = 0
    pbar = tqdm(tr_dl)
    for x in pbar:
        (target_idx, mask), (mats, cats, scalars) = x
        target_idx = target_idx.cuda()
        mask = mask.cuda().squeeze(1)
        for k in mats:
            mats[k] = mats[k].cuda()
        for k in cats:
            cats[k] = cats[k].cuda()
#         cats = {}
        scalars = scalars.cuda()
        for model, optimizer in zip(mlp_models, opts):
            model.zero_grad()
            pred = model(mats, cats, scalars)
            pred[mask.bool()] = -10000.0
#             pos = pred[torch.arange(pred.shape[0]).cuda(), target_idx]
#             pred[torch.arange(pred.shape[0]).cuda(), target_idx] = -1000.0
#             T = torch.clamp(-pos.unsqueeze(1) + pred + 1.5, min=0)
#             T = T.sum(-1) / (1e-8 + (T > 0).sum(-1).float())
#             loss_2 = T.mean()
#     #         loss = ce_loss + 0.01 * loss_2.mean()

#             loss = loss_2
            loss = lf(pred, target_idx)
            loss.mean().backward()
            optimizer.step()
        loss_ = 0.99 * loss_  + 0.01 * loss.detach().cpu().numpy()
        pbar.set_postfix({'loss:': "%0.4f" % loss_})

    model = mlp_models[0]
    if epoch < 10:
        continue
    mrr, hr = validate_models(mlp_models, val_ds)
    if mrr >= max_mrr:
        max_mrr = mrr
        torch.save(mlp_models, "val_no_shuffle_mlp_256_run7")
    print("[ITER %d] VAL MRR:" % (1 + epoch), mrr, "HR", hr)
    model = model.train()
    model.cuda()


100%|██████████| 1183/1183 [03:20<00:00,  5.90it/s, loss:=5.7635]


[ITER 11] VAL MRR: 0.18419919722875128 HR 0.25578317


100%|██████████| 1183/1183 [03:22<00:00,  5.84it/s, loss:=5.7267]


[ITER 12] VAL MRR: 0.18554280318639835 HR 0.25771907


100%|██████████| 1183/1183 [03:17<00:00,  6.00it/s, loss:=5.7002]


...

[ITER 28] VAL MRR: 0.19330828873111364 HR 0.26757008


100%|██████████| 1183/1183 [03:20<00:00,  5.90it/s, loss:=5.3298]


[ITER 29] VAL MRR: 0.1934222841050479 HR 0.26693296


100%|██████████| 1183/1183 [03:16<00:00,  6.03it/s, loss:=5.3080]


[ITER 30] VAL MRR: 0.19324543047499593 HR 0.267129


In [ ]:
def validate_models(models, val_ds, use_filter=True, skip_val_only=False, r=None):
    hr = []
    for model in models:
        model = model.eval()
    s = 0
    tot_mrr = []
    ii = 0
    pbar = tqdm(DataLoader(val_ds, batch_size=250, shuffle=False))
    for input in pbar:
        (target_idx, _), (ret_mat, ret_cat, ret_scalar) = input
        for k in ret_mat:
            ret_mat[k] = ret_mat[k].cuda()
        for k in ret_cat:
            ret_cat[k] = ret_cat[k].cuda()
        ret_scalar = ret_scalar.cuda()
        rret = []
        for model in models:
            ret = model.forward(ret_mat, ret_cat, ret_scalar,)
            rret.append(ret)
        ret = torch.stack(rret)
        ret = torch.mean(ret, 0)
#         if use_filter:
#             ret[:, ~rec_cand_mask] = -1000000.0
        ret[ret_mat['item_bow'].bool()] = -10000.0
        if r is not None:
            print(r.shape)
            ret[:, ~r] = -100000.0
        top_rec = (-ret).argsort(-1)[:, :100].detach().cpu()

#         if skip_val_only:
#         top_rec = top_rec[target_idx != 29999]
#         target_idx = target_idx[target_idx != 29999]

        mrr = (top_rec == target_idx.unsqueeze(1)).float().numpy()
        hitst = mrr[:, :5].sum(-1)
        mrr = (mrr / np.expand_dims(np.arange(1, 1 + 100), 0)).sum(-1)
        tot_mrr.extend(mrr.tolist())
        hr.extend(hitst)
        pbar.set_postfix({
            'mrr': "%0.4f" % np.mean(tot_mrr),
            'hr': "%0.4f" % np.mean(hr)

                         })
    for model in models:
        model = model.train()
    return tot_mrr, hr


In [ ]:
fname = "procseed_data_with_aug"

In [ ]:
mlp_models = torch.load("val_no_shuffle_mlp_256_run6")

In [ ]:
te_ds = joblib.load('%s/te_ds' % fname)

In [ ]:
mrr, hr = validate_models(mlp_models, te_ds)

100%|██████████| 164/164 [00:51<00:00,  3.20it/s, mrr=0.1902, hr=0.2671]


In [ ]:
np.mean(mrr), np.mean(hr)

(0.19015461356917332, 0.26714042)

In [ ]:
mrr, hr = validate_models(mlp_models, val_ds)

100%|██████████| 164/164 [00:50<00:00,  3.22it/s, mrr=0.1937, hr=0.2682]


In [ ]:
np.mean(mrr), np.mean(hr)

(0.19365474389197826, 0.26820722)